In [16]:
%load_ext autoreload
%autoreload 2
from specxploreImporting import importing
import matchms
from matchms.importing import load_from_mgf
from matchms.exporting import save_as_mgf
from ms2query.run_ms2query import download_zenodo_files, run_complete_folder
from ms2query.ms2library import create_library_object_from_one_dir
import os
import pandas as pd
import numpy as np
from typing import List
models_and_library_folder_pos = os.path.join("data_and_output", "models", "pos")
raw_experimental_mgf_filename = os.path.join("data_and_output", "test_data", "MonoisotopicMS2Feature_pos_mostIntense.mgf")
raw_standards_mgf_filename = os.path.join("data_and_output", "test_data", "phophe_pos.mgf")
processed_experimental_spectra_filename = os.path.join("data_and_output", "test_data", "spectra_experimental.mgf")
processed_standard_spectra_filename = os.path.join("data_and_output", "test_data", "spectra_standards.mgf")
csv_experimental_filename = os.path.join("data_and_output", "test_data", "test_data_ms2query_out_experimental.csv")
csv_standards_filename = os.path.join("data_and_output", "test_data", "test_data_ms2query_out_standard.csv")
scores_s2v_filename = os.path.join("data_and_output", "test_data", "s2v.npy")
scores_ms2ds_filename = os.path.join("data_and_output", "test_data", "ms2ds.npy")
scores_cos_filename = os.path.join("data_and_output", "test_data", "cos.npy")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
spectra_experimental = list(load_from_mgf(raw_experimental_mgf_filename))
spectra_standards = list(load_from_mgf(raw_standards_mgf_filename))
#(inspect.getmembers(spectra_experimental[0]))
#pectra_experimental[0].get("pepmass")


In [3]:
if False: # inspect new spectrum objects
    spec = spectra_experimental[0]
    spec = matchms.filtering.default_filters(spec)
    spec = matchms.filtering.normalize_intensities(spec)
    spec = matchms.filtering.select_by_relative_intensity(spec, 0.01, 1)
    spec = matchms.filtering.select_by_relative_intensity(spec, 0.01, 1)
    spec = matchms.filtering.repair_inchi_inchikey_smiles(spec)
    spec = matchms.filtering.harmonize_undefined_inchi(spec)
    spec = matchms.filtering.harmonize_undefined_inchikey(spec)
    spec = matchms.filtering.harmonize_undefined_smiles(spec)
    print(spec.get("precursor_mz"))
    print(spec.get("rtinseconds"))
    print(spec.peaks.mz)
    print(spec.peaks.intensities)
    print(len(spec.peaks.mz) == len(spec.peaks.intensities))
    print(spec.peaks.to_numpy)
    print(spec.metadata)

In [4]:
if False: # avoid adding to the spectra on each jupyter notebook rerun
    spectra_experimental = list(load_from_mgf(raw_experimental_mgf_filename))
    spectra_experimental = importing.clean_spectra(spectra_experimental)[0:20]
    save_as_mgf(spectra_experimental, processed_experimental_spectra_filename)

    spectra_standards = list(load_from_mgf(raw_standards_mgf_filename))
    spectra_standards = importing.clean_spectra(spectra_standards)[0:10]
    save_as_mgf(spectra_standards, processed_standard_spectra_filename)

In [5]:
spectra_experimental = list(load_from_mgf(processed_experimental_spectra_filename))
spectra_standards = list(load_from_mgf(processed_standard_spectra_filename))

In [6]:
if False:
    importing.run_single_file(ms2library, processed_standard_spectra_filename, str(csv_standards_filename))
    importing.run_single_file(ms2library, processed_experimental_spectra_filename, str(csv_experimental_filename))

In [7]:
classes_experimental = pd.read_csv(csv_experimental_filename)
classes_experimental = importing.expand_ms2query_results_table(classes_experimental, len(spectra_experimental))
classes_standards = pd.read_csv(csv_standards_filename)
classes_standards = importing.expand_ms2query_results_table(classes_standards, len(spectra_standards))

In [8]:
classes_experimental["is_standard"] = False
classes_experimental["exp_metadata"] = None
classes_standards["is_standard"] = True
classes_standards["standard_metadata"] = None
all_class_table = pd.merge(classes_experimental, classes_standards, how = 'outer').reset_index()
all_class_table["specxplore_id"] = all_class_table.index
all_class_table

,index,query_spectrum_nr,ms2query_model_prediction,precursor_mz_difference,precursor_mz_query_spectrum,precursor_mz_analog,inchikey,spectrum_ids,analog_compound_name,retention_time,...,cf_subclass,cf_direct_parent,npc_class_results,npc_superclass_results,npc_pathway_results,source_spectrum_mgf_iloc,is_standard,exp_metadata,standard_metadata,specxplore_id
0,0,1,0.3775,123.0081,105.0339,228.042,ZCGLNFAUVVHTQO,276075,3-[(4-chlorobenzoyl)amino]propanoic acid,NaN,...,"Amino acids, peptides, and analogues",Beta amino acids and derivatives,NaN,NaN,NaN,0,False,None,NaN,0
1,1,2,0.7120,0.0007,113.0237,113.023,IHCCAYCGZOLTEU,303482,3-FUROIC ACID,NaN,...,Furoic acid and derivatives,Furoic acids,NaN,NaN,NaN,1,False,None,NaN,1
2,2,3,0.7416,0.0001,120.0809,120.081,LPAGFVYQRIESJQ,313913,Indolin,NaN,...,Indolines,Indolines,Carboline alkaloids,Tryptophan alkaloids,Alkaloids,2,False,None,NaN,2
3,3,4,0.7399,0.0006,122.0966,122.096,UFFBMTHBGFGIHF,274723,"2,6-Dimethylaniline",NaN,...,Xylenes,m-Xylenes,NaN,NaN,NaN,3,False,None,NaN,3
4,4,5,0.4090,310.2715,125.0235,435.295,SCABKEBYDRTODC,265703,bis[2-(2-butoxyethoxy)ethyl] hexanedioate,NaN,...,Fatty acid esters,Fatty acid esters,Triacylglycerols,Glycerolipids,Fatty acids,4,False,None,NaN,4
5,5,6,0.8399,0.0004,127.0504,127.050,RWQNBRDOKXIBIV,303494,THYMINE,NaN,...,Pyrimidines and pyrimidine derivatives,Hydroxypyrimidines,Purine alkaloids; Pyridine alkaloids,Pseudoalkaloids (transamidation); Nicotinic ac...,Alkaloids,5,False,None,NaN,5
6,6,7,0.7312,0.0002,136.0618,136.062,GFFGJBXGBJISGV,57697,adenine,NaN,...,Purines and purine derivatives,6-aminopurines,Purine alkaloids,Pseudoalkaloids (transamidation),Alkaloids,6,False,None,NaN,6
7,7,8,0.6237,0.0001,143.0339,143.034,PCSKKIUURRTAEM,306586,5-HYDROXYMETHYL-2-FURANCARBOXYLIC ACID,NaN,...,NaN,NaN,NaN,NaN,NaN,7,False,None,NaN,7
8,8,9,0.7119,0.0006,144.0656,144.065,OYIFNHCXNCRBQI,28452,AMINOADIPATE,NaN,...,"Amino acids, peptides, and analogues",Alpha amino acids,Aminoacids,Small peptides,Amino acids and Peptides,8,False,None,NaN,8
9,9,10,0.6202,0.0006,144.0656,144.065,XLBVNMSMFQMKEY,28339,N-METHYLGLUTAMATE,NaN,...,"Amino acids, peptides, and analogues",Glutamic acid and derivatives,Aminoacids,Small peptides,Amino acids and Peptides,9,False,None,NaN,9


In [9]:
all_spectra = spectra_experimental + spectra_standards # list addition

[ 51.02379  53.0393   70.08131  72.93778  77.03909  78.03838  79.05466
  82.55311  90.94814  95.04961 103.05476 105.02575 105.03387 105.04509
 105.07024 114.49272]
[0.03       0.21578947 0.01315789 0.06315789 0.12105263 0.01263158
 0.02631579 0.01263158 0.02315789 1.         0.01526316 0.01842105
 0.57894737 0.27368421 0.06842105 0.01210526]


In [10]:
scores_s2v = importing.compute_similarities_s2v(all_spectra, models_and_library_folder_pos)
scores_cos = importing.compute_similarities_cosine(all_spectra, cosine_type="ModifiedCosine")
scores_ms2ds = importing.compute_similarities_ms2ds(all_spectra, models_and_library_folder_pos)

2023-04-12 11:27:58,051:WARNING:matchms:ModifiedCosine:Precursor_mz must be of type int or float. Apply 'add_precursor_mz' filter first.
2023-04-12 11:27:58,052:WARNING:matchms:ModifiedCosine:Precursor_mz must be of type int or float. Apply 'add_precursor_mz' filter first.
2023-04-12 11:27:58,539:WARNING:matchms:ModifiedCosine:Precursor_mz must be of type int or float. Apply 'add_precursor_mz' filter first.
2023-04-12 11:27:58,540:WARNING:matchms:ModifiedCosine:Precursor_mz must be of type int or float. Apply 'add_precursor_mz' filter first.
2023-04-12 11:27:58,540:WARNING:matchms:ModifiedCosine:Precursor_mz must be of type int or float. Apply 'add_precursor_mz' filter first.
2023-04-12 11:27:58,540:WARNING:matchms:ModifiedCosine:Precursor_mz must be of type int or float. Apply 'add_precursor_mz' filter first.
2023-04-12 11:27:58,541:WARNING:matchms:ModifiedCosine:Precursor_mz must be of type int or float. Apply 'add_precursor_mz' filter first.
2023-04-12 11:27:58,541:WARNING:matchms:M

Calculating vectors of reference spectrums: 100%|██████████| 30/30 [00:00<00:00, 38.60it/s]


In [11]:
k_values = [3, 5, 10]
random_seeds = [int(np.random.randint(0,1000)) for _ in k_values]
similarities = scores_ms2ds
distances = importing.convert_similarity_to_distance(similarities)
kmedoid_list = importing.run_kmedoid_grid(distances, k_values, random_seeds)
importing.render_kmedoid_fitting_results_in_browser(kmedoid_list)

In [12]:
perplexity_values = [x for x in range(5, 25, 5)]
random_seeds = [int(np.random.randint(0,1000)) for _ in perplexity_values]
tsne_list_pos = importing.run_tsne_grid(distances, perplexity_values, random_seeds)
importing.render_tsne_fitting_results_in_browser(tsne_list_pos)

In [32]:
%load_ext autoreload
%autoreload 2
import specxplore.specxplore_data
from specxplore.specxplore_data import specxplore_data, Spectrum
is_standard = np.array(all_class_table["is_standard"])
spec_classes = all_class_table[['cf_kingdom', 'cf_superclass', 'cf_class', 'cf_subclass',
       'cf_direct_parent', 'npc_class_results', 'npc_superclass_results',
       'npc_pathway_results']]
mz = [spec.get("precursor_mz") for spec in all_spectra]
# KmedoidGridEntry(k, cluster_assignments, score, random_states[idx])
kclass_table = pd.DataFrame()
for elem in kmedoid_list:
    kclass_table[elem.k] = elem.cluster_assignments
classes = pd.concat([spec_classes, kclass_table], axis=1)
tsnedf = pd.DataFrame({"x" : tsne_list_pos[2].x_coordinates, "y" : tsne_list_pos[2].y_coordinates})
specxplore_id = np.array(all_class_table["specxplore_id"])

tsnedf["is_standard"] = is_standard
tsnedf["id"] = specxplore_id

def convert_matchms_spectra_to_specxplore_spectrum(spectra = List[matchms.Spectrum]) -> List[Spectrum]:
  spectra_converted = [
      Spectrum(spec.peaks.mz, float(spec.get("precursor_mz")), idx, spec.peaks.intensities) 
      for idx, spec in enumerate(all_spectra)]
  return spectra_converted

spectra_converted = convert_matchms_spectra_to_specxplore_spectrum(all_spectra)
test_data_specxplore = specxplore_data(
  scores_ms2ds,scores_s2v, scores_cos, tsnedf,  classes,
  is_standard, spectra_converted, mz, specxplore_id)

import pickle
with open('data_and_output/test_data/test_case_specxplore5.pickle', 'wb') as file:
  pickle.dump(test_data_specxplore, file)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
